In [1]:
import sys
import warnings
import torch
import torch_geometric.transforms as T
from utils import Complete
from dataset_no_rdkit import TencentAlchemyDataset
from trainer import Trainer
from model import RGNN

In [7]:
warnings.filterwarnings("ignore")
task = 2
option = {
    'train_epoch': 2,
    'train_batch': 2,
    'task': task,  # [0~11]
    'lr': 0.001,
    'lr_scheduler_patience': 3,
    'parallel': False,
    'cuda_devices': [0, 1, 2, 3],  # works when parallel=True
    'early_stop_patience': 10,  # -1 for no early stop
}


class SpecifyTarget(object):
    def __call__(self, data):
        data.y = data.y[0, option['task']].view(-1)
        return data


print('Load Dataset...')
transform = T.Compose([SpecifyTarget(), Complete(), T.Distance(norm=True)])
path = '../alchemy_dataset'
dataset = TencentAlchemyDataset(root=path, transform=transform).shuffle()

print('Split the dataset...')
one_tenth = len(dataset) // 10000
test_dataset = dataset[: one_tenth]
valid_dataset = dataset[one_tenth: one_tenth * 2]
train_dataset = dataset[one_tenth * 2: one_tenth * 3]

Load Dataset...
Split the dataset...


In [2]:
print('Load Dataset...')
transform = T.Compose([Complete(), T.Distance(norm=True)])
path = '../alchemy_dataset'
dataset = TencentAlchemyDataset(root=path, transform=transform).shuffle()

print('Split the dataset...')
one_tenth = len(dataset) // 10000
test_dataset = dataset[: one_tenth]
valid_dataset = dataset[one_tenth: one_tenth * 2]
train_dataset = dataset[one_tenth * 2: one_tenth * 3]
train_dataset[0].y

Load Dataset...
Split the dataset...


tensor([[ 0.6107,  0.7344, -0.2219,  0.2685, -0.1669,  0.2685, -0.0887,  0.2685,
          0.2685, -0.3893, -0.2727, -0.0342]])

In [9]:
print('Training init...')
model = RGNN()
trainer = Trainer(option, model, \
                  train_dataset, valid_dataset, test_dataset)

Training init...
	train_epoch:2
	train_batch:2
	task:2
	lr:0.001
	lr_scheduler_patience:3
	parallel:False
	cuda_devices:[0, 1, 2, 3]
	early_stop_patience:10
save id: dhga
train set num:9 valid set num:9 test set num: 9
Total Parameters:2865729


In [10]:
trainer.train()

Training start...


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

	batch 0 training loss: 0.34023 Time elapsed 0.00 hrs (0.1 mins)
Epoch:0 trn_loss:1.31765 val_loss:0.89525 Time elapsed 0.00 hrs (0.2 mins)
Model saved at epoch 0


 50%|██████████████████████████████████████████                                          | 1/2 [00:12<00:12, 12.44s/it]

	batch 0 training loss: 0.57287 Time elapsed 0.00 hrs (0.3 mins)
Epoch:1 trn_loss:1.27549 val_loss:0.85358 Time elapsed 0.01 hrs (0.4 mins)
Model saved at epoch 1


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:22<00:00, 11.20s/it]


Model saved at epoch 1


In [11]:
trainer.load_best_ckpt()

The best ckpt is D:\Jupyter\RGNN\src_v7.4_alchemy\ckpt\ckpts_task2_dhga/task2_1_1.27549_0.85358.ckpt
Ckpt loading: D:\Jupyter\RGNN\src_v7.4_alchemy\ckpt\ckpts_task2_dhga/task2_1_1.27549_0.85358.ckpt


In [12]:
trainer.valid_iterations(mode='test')

Test loss: 0.89417


0.8941701054573059